In [121]:
import pandas as pd

# Importation des données nettoyées
donnees = pd.read_csv("data_nettoyer.csv")

# Aperçu des données
print(donnees.head())
print(donnees.info())



   Âge    Sexe     IMC  Enfants Fumeur     Région        Frais
0   19  female  27.900        0    yes  southwest  16884.92400
1   18    male  33.770        1     no  southeast   1725.55230
2   28    male  33.000        3     no  southeast   4449.46200
3   33    male  22.705        0     no  northwest  21984.47061
4   32    male  28.880        0     no  northwest   3866.85520
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328 entries, 0 to 1327
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Âge      1328 non-null   int64  
 1   Sexe     1328 non-null   object 
 2   IMC      1328 non-null   float64
 3   Enfants  1328 non-null   int64  
 4   Fumeur   1328 non-null   object 
 5   Région   1328 non-null   object 
 6   Frais    1328 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 72.8+ KB
None


In [122]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error

# Séparation des données
X = donnees.drop(columns=['Frais'])
y = donnees['Frais']

# Encodage des variables catégoriques pour éviter les erreurs
X = pd.get_dummies(X, drop_first=True)

# Division en données d'entraînement et de test (80 % / 20 %)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dummy model
dummy_model = DummyRegressor(strategy="mean")
dummy_model.fit(X_train, y_train)

# Prédictions et évaluation
y_pred_dummy = dummy_model.predict(X_test)

# Évaluation des performances
r2 = r2_score(y_test, y_pred_dummy)
rmse = root_mean_squared_error(y_test, y_pred_dummy)

print("Dummy Model - R² :", r2)
print("Dummy Model - RMSE :", rmse)


Dummy Model - R² : -0.0001034630945579984
Dummy Model - RMSE : 11790.382185510607


In [123]:
import numpy as np

# Transformation logarithmique pour y si nécessaire
y = np.log1p(donnees['Frais'])  # log(1 + y) pour éviter les problèmes avec des zéros


In [124]:
random_state = 42


In [125]:
# Évaluation des modèles
from sklearn.metrics import root_mean_squared_error, r2_score

resultats = {}

for nom, pipeline in modeles.items():
    # Entraînement
    pipeline.fit(X_train, y_train)
    
    # Prédiction
    y_pred = pipeline.predict(X_test)
    
    # Évaluation
    r2 = r2_score(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    
    # Stockage des résultats
    resultats[nom] = {'R²': r2, 'RMSE': rmse}
    
    print(f"{nom} - R² : {r2:.4f}, RMSE : {rmse:.4f}")

# Résultats dans un DataFrame pour comparaison
resultats_df = pd.DataFrame(resultats).T
print("\nRésumé des performances :")
print(resultats_df)


Régression Linéaire - R² : 0.1579, RMSE : 10818.9214
Lasso - R² : 0.1579, RMSE : 10818.9730
Ridge - R² : 0.1579, RMSE : 10819.0599
ElasticNet - R² : 0.1559, RMSE : 10832.0386

Résumé des performances :
                           R²          RMSE
Régression Linéaire  0.157913  10818.921422
Lasso                0.157905  10818.972987
Ridge                0.157891  10819.059912
ElasticNet           0.155870  10832.038600


In [126]:
from sklearn.preprocessing import PolynomialFeatures

# Ajout de PolynomialFeatures dans le pipeline (pour ElasticNet par exemple)
polynomial_pipeline = Pipeline(steps=[
    ('preprocessing', preprocesseur),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=random_state))
])


In [127]:
from sklearn.model_selection import GridSearchCV

# Paramètres à tester pour ElasticNet
parametres = {
    'poly__degree': [1, 2, 3],
    'model__alpha': [0.01, 0.1, 1.0],
    'model__l1_ratio': [0.2, 0.5, 0.8]
}

# GridSearchCV
grid_search = GridSearchCV(polynomial_pipeline, parametres, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur RMSE :", np.sqrt(-grid_search.best_score_))


Meilleurs paramètres : {'model__alpha': 0.1, 'model__l1_ratio': 0.5, 'poly__degree': 1}
Meilleur RMSE : 11426.230022966405


In [128]:
# Récupérer les coefficients pour ElasticNet
meilleur_modele = grid_search.best_estimator_
coefficients = meilleur_modele.named_steps['model'].coef_

# Associer les coefficients aux noms des variables
colonnes_finales = meilleur_modele.named_steps['preprocessing'].transformers_[0][1].get_feature_names_out()
coeff_df = pd.DataFrame({'Variable': colonnes_finales, 'Coefficient': coefficients})

print("\nVariables les plus importantes :")
print(coeff_df.sort_values(by='Coefficient', ascending=False).head(10))



Variables les plus importantes :
  Variable  Coefficient
0      Âge  3102.034565
1      IMC  1730.553726
2  Enfants   768.498247


In [129]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

# Définir les hyperparamètres à tester
param_grid = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'l1_ratio': [0.1, 0.5, 0.9]
}

# Initialiser le modèle
elastic_net = ElasticNet(max_iter=10000)

# Initialiser la recherche sur grille
grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Entraîner la recherche sur les données d'entraînement
grid_search.fit(X_train, y_train)

# Meilleurs hyperparamètres
print("Meilleurs hyperparamètres :", grid_search.best_params_)

# Meilleur score
print("Meilleur score R² :", grid_search.best_score_)


Meilleurs hyperparamètres : {'alpha': 0.1, 'l1_ratio': 0.9}
Meilleur score R² : 0.7402415684619652
